In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 144542 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.18-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.18-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
from drive.ML_Study import * # 让driver连接云盘的ML_Study

In [0]:
# For console 
function ClickConnect(){
  console.log("Working"); 
  document
    .querySelector("#top-toolbar > colab-connect-button")
    .shadowRoot
    .querySelector("#connect")
    .click()
}
 
setInterval(ClickConnect,60000)

SyntaxError: ignored

In [0]:
pip install transformers

     |████████████████████████████████| 552kB 2.7MB/s 
     |████████████████████████████████| 3.7MB 14.9MB/s 
     |████████████████████████████████| 1.0MB 36.0MB/s 
     |████████████████████████████████| 870kB 31.3MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=eb6b43f1252156531ee07d68cb44e797bc47d266ae5d2722d8dd370f8613651d
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
import json
import numpy as np
from tqdm import tqdm
import os
import tensorflow as tf
from transformers import *

In [0]:
print('Lodaing BERT')
# 最好使用自己下载的文件，否则可能会因为连不上下载文件的服务器而报错
pretrainedModel_path = 'drive/ML_Study/bert-base-chinese'
tokenizer = BertTokenizer.from_pretrained(os.path.join(pretrainedModel_path, 'vocab.txt'))
config = BertConfig.from_json_file(os.path.join(pretrainedModel_path, 'config.json'))
print('==========END==========')

Lodaing BERT


Calling BertTokenizer.from_pretrained() with the path to a single file or url is deprecated


==========END==========


In [0]:
# 生成新的SPO
# def load_data(path):
#     D = []
#     with open(path, encoding='utf-8') as f:
#         for l in f.readlines():
#             l = json.loads(l)
#             D.append({
#                 'text': l['text'],
#                 'spo_list': [
#                     (spo['subject'], spo['predicate'], spo['object']['@value'])
#                     for spo in l['spo_list']
#                 ]
#             })
#     return D

def load_data(filename):
    D = []
    with open(filename, encoding='utf-8') as f:
        for l in f:
            l = json.loads(l)
            D.append({
                'text': l['text'],
                'spo_list': [(spo['subject'], spo['predicate'], spo['object'])
                             for spo in l['spo_list']]
            })
    return D

# 得到对应值的初始下标
def search(pattern, sequence):
    n = len(pattern)
    for i in range(len(sequence)):
        if sequence[i:i + n] == pattern:
            return i
    return -1

In [0]:
print('Loading Data')
train_data = load_data('drive/ML_Study/B4K_Data/train_data.json')
# train_data = load_data('drive/ML_Study/Baidu_ERE_Data/sample_data.json')
valid_data = load_data('drive/ML_Study/B4K_Data/dev_data.json')
# test_data = load_data('./data/example.test')
print('=========END==========')

Loading Data
=========END==========


In [0]:
# category mapping
# def get_mapping(mapping_path):
#     predicate2id, id2predicate = {}, {}
#     with open(mapping_path, encoding='utf-8') as f:
#         for l in f.readlines():
#             l = json.loads(l)
#             if l['predicate'] not in predicate2id:
#                 id2predicate[len(predicate2id)] = l['predicate']
#                 predicate2id[l['predicate']] = len(predicate2id)
#     return predicate2id, id2predicate
    
# mapping_path = 'drive/ML_Study/Baidu_ERE_Data/schema.json'
# predicate2id, id2predicate = get_mapping(mapping_path)
predicate2id, id2predicate = {}, {}

with open('drive/ML_Study/B4K_Data/all_50_schemas') as f:
    for l in f:
        l = json.loads(l)
        if l['predicate'] not in predicate2id:
            id2predicate[len(predicate2id)] = l['predicate']
            predicate2id[l['predicate']] = len(predicate2id)

In [0]:
def get_input(data,tokenizer,predicate2id,maxlen=128):
        token_ids_t1, segment_ids_t1, position_ids_t1= [], [], [] # X for S
        token_ids_t2, segment_ids_t2, position_ids_t2= [], [], [] # X for PO
        subject_labels, object_labels = [], []# Y
        for d in data:
            # 针对一个text
            token_id, segment_id, position_id = tokenizer.encode_plus(d['text'], max_length=maxlen,pad_to_max_length=True).values()

            SPOes = {}            
            for s, p, o in d['spo_list']:
                sw = s
                s = tokenizer.encode(s)[1:-1]# 101 是 CLS 102 是ESP 这里只是为了得到s内部值的ID
                p = predicate2id[p] # 对category进行映射得到对应的ID
                o = tokenizer.encode(o)[1:-1] 
                s_idx = search(s, token_id) # 对text进行映射
                o_idx = search(o, token_id)
                if s_idx != -1 and o_idx != -1:
                    s = (s_idx, s_idx + len(s) - 1) # s 为 subject 的位置（始终态）
                    o = (o_idx, o_idx + len(o) - 1, p) # 包含了 object 和 predicate
                    if s not in SPOes:
                        SPOes[s] = []
                        # forT2，在text_a的基础上加上text_a(subject)
                        token_idv2, segment_idv2, position_idv2 = tokenizer.encode_plus(text=d['text'], text_pair=sw*4, max_length=maxlen,pad_to_max_length=True).values()
                        token_ids_t2.append(token_idv2)
                        segment_ids_t2.append(segment_idv2)
                        position_ids_t2.append(position_idv2)
                    SPOes[s].append(o) # 可一对多
                    
            if SPOes:
                # Model 1
                # subject label
                subject_label = np.zeros((len(token_id), 2)) #两层
                Ss = SPOes.keys()
                for s in Ss:
                    subject_label[s[0], 0] = 1 # label第一层保存初始位置
                    subject_label[s[1], 1] = 1 # 第二层保存末态位置
                token_ids_t1.append(token_id)
                segment_ids_t1.append(segment_id)
                position_ids_t1.append(position_id)
                subject_labels.append(subject_label) 
                
                # Model 2
                # object和predicate label
                for s in Ss:
                    object_label = np.zeros((len(token_id), len(predicate2id), 2)) 
                    for o in SPOes.get(s, []): # 得到对应的po 若sub_id不存在，则返回空[]
                        object_label[o[0], o[2], 0] = 1 # o[2] 是predicate
                        object_label[o[1], o[2], 1] = 1 
                    object_labels.append(object_label)
                    # 其余与 subject 预测数据的 label处理类似,只不过这里label也同时标上了
        print('Converting to tensor.....')
        return (
            tf.convert_to_tensor(tf.constant(token_ids_t1)),
            tf.convert_to_tensor(tf.constant(segment_ids_t1)),
            tf.convert_to_tensor(tf.constant(position_ids_t1)),
            tf.convert_to_tensor(tf.constant(token_ids_t2)), 
            tf.convert_to_tensor(tf.constant(segment_ids_t2)), 
            tf.convert_to_tensor(tf.constant(position_ids_t2)),
            tf.convert_to_tensor(tf.constant(subject_labels)), 
            tf.convert_to_tensor(tf.constant(object_labels))
        )

In [0]:
print('Contructing Input')
token_ids_t1, segment_ids_t1, position_ids_t1,token_ids_t2, segment_ids_t2, position_ids_t2, subject_labels, object_labels = get_input(train_data,tokenizer,predicate2id)
print('=========END==========')

Contructing Input
Converting to tensor.....


In [0]:
token_ids_t1_valid, segment_ids_t1_valid, position_ids_t1_valid,token_ids_t2_valid, segment_ids_t2_valid, position_ids_t2_valid, subject_labels_valid, object_labels_valid = get_input(valid_data,tokenizer,predicate2id)

In [0]:
print(len(token_ids_t1) == len(train_data))
print(len(token_ids_t1) == len(subject_labels))
print(len(token_ids_t2) == len(object_labels))
# 数据集中有空的spolist

In [0]:
def extract_SPOes(text):
    """
    抽取输入text所包含的三元组
    """
    tokens = tokenizer.tokenize(text, max_length=maxlen)
    token_ids, segment_ids = tokenizer.encode(text, max_length=maxlen)
    # 抽取subject
    subject_preds = subject_model.predict([[token_ids], [segment_ids]])
    start = np.where(subject_preds[0, :, 0] > 0.6)[0]  # 输出坐标
    end = np.where(subject_preds[0, :, 1] > 0.5)[0]
    subjects = []
    for i in start:
        j = end[end >= i]
        if len(j) > 0:
            j = j[0]
            subjects.append((i, j))
    if subjects:
        SPOes = []
        # 根据s内存在的subject个数进行复制
        token_ids = np.repeat([token_ids], len(subjects), 0) # axis = 0 增加行
        segment_ids = np.repeat([segment_ids], len(subjects), 0)
        subjects = np.array(subjects)
        # 传入subject，抽取object和predicate
        object_preds = object_model.predict([token_ids, segment_ids, subjects]) 
        for subject, object_pred in zip(subjects, object_preds):
            start = np.where(object_pred[:, :, 0] > 0.6) # 二维坐标
            end = np.where(object_pred[:, :, 1] > 0.5)
            for _start, predicate1 in zip(*start): 
                for _end, predicate2 in zip(*end):
                    if _start <= _end and predicate1 == predicate2:
                        SPOes.append((subject, predicate1, (_start, _end)))
                        break
        return [
            (
                tokenizer.decode(token_ids[0, s[0]:s[1] + 1], tokens[s[0]:s[1] + 1]),
                id2predicate[p],
                tokenizer.decode(token_ids[0, o[0]:o[1] + 1], tokens[o[0]:o[1] + 1])
            ) for s, p, o in spoes
        ]
    else:
        return []

In [0]:
class SPO(tuple):
    def __init__(self, spo):
        self.spox = (
            tuple(tokenizer.tokenize(spo[0])),
            spo[1],
            tuple(tokenizer.tokenize(spo[2])),
        )

    def __hash__(self):
        return self.spox.__hash__()

    def __eq__(self, spo):
        return self.spox == spo.spox

In [0]:
def evaluate(data):
    X, Y, Z = 1e-10, 1e-10, 1e-10
    f = open('dev_pred.json', 'w', encoding='utf-8')
    pbar = tqdm()
    for d in data:
        R = set([SPO(spo) for spo in extract_SPOes(d['text'])]) # spo 是 tup！
        T = set([SPO(spo) for spo in d['spo_list']])
        X += len(R & T)
        Y += len(R)
        Z += len(T)
        f1, precision, recall = 2 * X / (Y + Z), X / Y, X / Z
        pbar.update()
        pbar.set_description('f1: %.5f, precision: %.5f, recall: %.5f' %
                             (f1, precision, recall))
        s = json.dumps(
            {
                'text': d['text'],
                'spo_list': list(T),
                'spo_list_pred': list(R),
                'new': list(R - T),
                'lack': list(T - R),
            },
            ensure_ascii=False,
            indent=4)
        f.write(s + '\n')
    pbar.close()
    f.close()
    return f1, precision, recall

In [0]:
class Evaluator(tf.keras.callbacks.Callback):
    """
    评估和保存模型
    """
    def __init__(self):
        self.best_val_f1 = 0.

    def on_epoch_end(self, epoch, logs=None):
        optimizer.apply_ema_weights()
        f1, precision, recall = evaluate(valid_data)
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
#             train_model.save_weights('./save/best_model.weights')
        optimizer.reset_old_weights()
        print('f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f\n' %
              (f1, precision, recall, self.best_val_f1))

In [0]:
class Subject_Model(TFBertPreTrainedModel):
    # def dummy_inputs(self):
    #     """ Dummy inputs to build the network.

    #     Returns:
    #         tf.Tensor with dummy inputs
    #     """
    #     return {"input_ids": tf.constant(DUMMY_INPUTS)}
    
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = TFBertMainLayer(config, name="bert")
        # self.dense01 = tf.keras.layers.Dense(units = 1024, activation=tf.nn.relu)
        self.dense02 = tf.keras.layers.Dense(units = 128*2, activation=tf.nn.softmax)

    def call(self, inputs, **kwargs):
        _, x = self.bert(inputs, **kwargs)
        # x = self.dense01(x)
        x = self.dense02(x)
        x = tf.keras.layers.Lambda(lambda x: x**2)(x)
        output = tf.reshape(x, (-1,128,2))
        return output

class Object_Model(TFBertPreTrainedModel):
    # def dummy_inputs(self):
    #     """ Dummy inputs to build the network.

    #     Returns:
    #         tf.Tensor with dummy inputs
    #     """
    #     return {"input_ids": tf.constant(DUMMY_INPUTS)}
    
    def __init__(self, config, *inputs, **kwargs):
        super().__init__(config, *inputs, **kwargs)
        self.bert = TFBertMainLayer(config, name="bert")
        # self.dense01 = tf.keras.layers.Dense(units = , activation=tf.nn.relu)
        self.dense02 = tf.keras.layers.Dense(units = 128*48*2, activation=tf.nn.sigmoid)

    def call(self, inputs, **kwargs):
        _, x = self.bert(inputs, **kwargs)
        # x = self.dense01(x)
        x = self.dense02(x)
        output = tf.reshape(x, (-1,128,48,2))
        return output

In [0]:
# train_dataset  = tf.data.Dataset.from_tensor_slices((train_x, train_y))
# train_dataset = train_dataset.shuffle(buffer_size=3000)    
# train_dataset = train_dataset.batch(32)
# train_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

# valid_dataset = tf.data.Dataset.from_tensor_slices((valid_x, valid_y))
# valid_dataset = train_dataset.shuffle(buffer_size=3000)    
# valid_dataset = train_dataset.batch(32)
# valid_dataset = train_dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [0]:
token_ids_t1[:7000].shape

In [0]:
def myloss(y_true, y_pred):
	return tf.reduce_sum(tf.keras.losses.binary_crossentropy(y_true,y_pred),axis=1)

In [0]:
config = BertConfig.from_json_file(os.path.join(pretrainedModel_path, 'config.json'))
subject_model = Subject_Model.from_pretrained(pretrainedModel_path,config=config)

subject_model.compile(
           optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), 
           loss=myloss, 
           metrics=[tf.keras.metrics.BinaryAccuracy()]) # accuracy没有好的 所以随便选了一个

In [0]:
subject_model.summary()

Model: "subject__model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
bert (TFBertMainLayer)       multiple                  102267648 
_________________________________________________________________
dense_1 (Dense)              multiple                  196864    
Total params: 102,464,512
Trainable params: 102,464,512
Non-trainable params: 0
_________________________________________________________________


In [0]:
# subject_model.fit(x={'input_ids':token_ids_t1[:7000], 'attention_mask':position_ids_t1[:7000], 'token_type_ids':segment_ids_t1[:7000]},
#           y=subject_labels[:7000],
#           epochs=6,
#           validation_steps=7,
#           validation_data=([token_ids_t1[7000:],position_ids_t1[7000:],segment_ids_t1[7000:]], subject_labels[7000:]),
#           batch_size=16
#            )
subject_model.fit(x={'input_ids':token_ids_t1, 'attention_mask':position_ids_t1, 'token_type_ids':segment_ids_t1},
          y=subject_labels,
          epochs=6,
          validation_steps=7,
          validation_data=([token_ids_t1_valid, position_ids_t1_valid, segment_ids_t1_valid], subject_labels_valid),
          batch_size=16
           )

Epoch 1/6
438/438 [==============================] - 339s 775ms/step - loss: 11.8944 - binary_accuracy: 0.9840 - val_loss: 6.2265 - val_binary_accuracy: 0.9901
Epoch 2/6
438/438 [==============================] - 337s 769ms/step - loss: 6.0983 - binary_accuracy: 0.9903 - val_loss: 5.5866 - val_binary_accuracy: 0.9910
Epoch 3/6
438/438 [==============================] - 338s 771ms/step - loss: 5.6787 - binary_accuracy: 0.9909 - val_loss: 5.2607 - val_binary_accuracy: 0.9915
Epoch 4/6
438/438 [==============================] - 337s 769ms/step - loss: 5.3926 - binary_accuracy: 0.9911 - val_loss: 5.0390 - val_binary_accuracy: 0.9915
Epoch 5/6
438/438 [==============================] - 336s 767ms/step - loss: 5.1109 - binary_accuracy: 0.9913 - val_loss: 4.7980 - val_binary_accuracy: 0.9916
Epoch 6/6
438/438 [==============================] - 338s 771ms/step - loss: 4.8190 - binary_accuracy: 0.9914 - val_loss: 4.6360 - val_binary_accuracy: 0.9919


In [0]:
# tf.saved_model.save(subject_model, "drive/ML_Study/saved_model/subject_model_200330v2")
# subject_model = tf.saved_model.load("drive/ML_Study/saved_model/subject_model_200330")

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: drive/ML_Study/saved_model/subject_model_200330/assets


In [0]:
def extract_Ss(text):
    maxlen=128
    tokens = tokenizer.tokenize(text, max_length=maxlen)
    a,b,c= tokenizer.encode_plus(text,max_length=maxlen,pad_to_max_length=True).values()

    _a = tf.convert_to_tensor(tf.constant([a])) # 输入model用
    _b = tf.convert_to_tensor(tf.constant([b]))
    _c = tf.convert_to_tensor(tf.constant([c]))
    token_id = np.array(a) # decode用
    # 抽取subject
    input_text={'input_ids':_a, 'attention_mask':_b, 'token_type_ids':_c}
    subject_preds = subject_model.predict(input_text)
    subject_preds = tf.nn.softmax(subject_preds)
    print(subject_preds)
    # 输出坐标
    start = np.where(subject_preds[0, :, 0] > 0.5)[0] 
    end = np.where(subject_preds[0, :, 1] > 0.5)[0]
    subjects = []
    
    for i in start:
        j = end[end >= i]
        if len(j) > 0:
            j = j[0]
            subjects.append((i, j))
    # subjects=[(1,3)]
    if subjects:
        rt = []
        for s in subjects:
            sub = tokenizer.decode(token_id[s[0]:s[1] + 1])
            rt.append(sub)
        return rt
    else:
        return []

In [0]:
text1 = '抗美援朝胜利后，张仲先将军回国与杨郁芬女士喜结连理' # 张仲先
text = "2013年4月21日，由潇湘电影集团有限公司与四川省阿坝州金川县政府联合出品的电影《马奈的新娘》在北京首映，影片反映嘉绒藏族民俗文化和藏区自然风光，以东女国文化为核心，展示了国家非物质文化遗产——宫廷歌舞马奈锅庄"
text = '亚伦的配音则是下野纮（又是悠哥）'

rt = extract_Ss(text)
rt = np.array(rt)
print(rt)
# tokens = tokenizer.tokenize(text, max_length=256)
# _tokens = np.array(tokens)
# a,b,c= tokenizer.encode_plus(text=s1,max_length=256,pad_to_max_length=True).values()
# _a = np.array(a)
# sub = tokenizer.decode(_a[s[0]:s[1] + 1])
# print(sub)

tf.Tensor(
[[[0.4999351  0.5000649 ]
  [0.51770717 0.48229286]
  [0.49367487 0.5063251 ]
  [0.50605714 0.49394283]
  [0.5088337  0.4911663 ]
  [0.512335   0.487665  ]
  [0.50033456 0.4996654 ]
  [0.503017   0.496983  ]
  [0.509054   0.49094602]
  [0.5070284  0.4929716 ]
  [0.5025168  0.49748313]
  [0.49627772 0.50372225]
  [0.49555278 0.5044472 ]
  [0.5062518  0.4937482 ]
  [0.49301806 0.50698197]
  [0.50545007 0.4945499 ]
  [0.49871972 0.50128025]
  [0.5004256  0.4995744 ]
  [0.49063855 0.50936145]
  [0.4979298  0.5020702 ]
  [0.4953353  0.50466466]
  [0.50044465 0.49955535]
  [0.50229204 0.49770796]
  [0.49797124 0.50202876]
  [0.4944411  0.50555885]
  [0.49557048 0.5044295 ]
  [0.49960262 0.5003974 ]
  [0.4992849  0.50071514]
  [0.49917552 0.50082445]
  [0.49445322 0.5055468 ]
  [0.5006062  0.4993938 ]
  [0.50126886 0.49873114]
  [0.500177   0.49982294]
  [0.502066   0.49793398]
  [0.5019876  0.4980124 ]
  [0.50014985 0.49985018]
  [0.49802637 0.5019736 ]
  [0.49821433 0.5017857 ]
 

In [0]:
a = np.zeros((4,7,2))
a[0,2,0] = 1
a[0,4,1] = 1
a[0,5,0] = 1
a[0,6,1] = 1
b = np.zeros((4,7,2))
start = np.where(a[0, :, 0] > 0.6)[0]
end = np.where(a[0, :, 1] > 0.5)[0]
# tf.reduce_sum(tf.keras.losses.binary_crossentropy(a, b),axis=1)
rt = np.array(myloss(a,b))
rt1 = np.array(tf.keras.losses.binary_crossentropy(a[0],a[1]))
print(rt)
print(rt1.shape)

[30.84989694 -0.         -0.         -0.        ]
(7,)


In [0]:
m = tf.keras.metrics.BinaryAccuracy()
r = m.update_state(a, b)
print(m.result().numpy())

0.9285714
